<a href="https://colab.research.google.com/github/hildazaqya/project-capstone/blob/main/firna/Foodection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
name = 'sayfirna miftahul jannah'
print(name)

sayfirna miftahul jannah


# Download Data from Kaggle

This data is collect mannually from many website, dan also in the data was captured using a smartphone camera.

In [3]:
#kaggle
!pip install kaggle

In [4]:
#this file from kaggle API personnal account
from google.colab import files

# Upload kaggle.json
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"smjayyy","key":"a86a230a085a77cff7a5851725589584"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
#link dataset in Kaggle.com
!kaggle datasets download -d alinesellwia/foodection-dataset

 99% 980M/994M [00:10<00:00, 118MB/s]
100% 994M/994M [00:10<00:00, 103MB/s]


In [7]:
#new directory for stores temporary data
!mkdir data

In [8]:
#unzip the data
!unzip -q foodection-dataset.zip -d data/datasetfull

#Install Library

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os

#Set the directory to easily navigate to the generator

In [29]:
base_dir= '/content/data/datasetfull/Foodection Dataset'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/Train'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/Test'))

Contents of base directory:
['Train', 'Test']

Contents of train directory:
['Jeruk Segar', 'Wortel Segar', 'Wortel Segar Sedang ', 'Tomat Busuk', 'Jeruk Segar Sedang', 'Jeruk Busuk', 'Tomat Segar sedang', 'Wortel Busuk', 'Tomat Segar']

Contents of validation directory:
['Jeruk Segar', 'Wortel Segar', 'Wortel Segar Sedang', 'Tomat Busuk', 'Jeruk Segar Sedang', 'Tomat Segar Sedang', 'Jeruk Busuk', 'Wortel Busuk', 'Tomat Segar']


In [34]:
train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')

# Specify class of the directory train
train_jeruksegar_dir = os.path.join(train_dir, 'Jeruk Segar')
train_wortelsegar_dir = os.path.join(train_dir, 'Wortel Segar')
train_wortelsegarsedang_dir = os.path.join(train_dir, 'Wortel Segar Sedang ')
train_tomatbusuk_dir = os.path.join(train_dir, 'Tomat Busuk')
train_jeruksegarsedang_dir = os.path.join(train_dir, 'Jeruk Segar Sedang')
train_jerukbusuk_dir = os.path.join(train_dir, 'Jeruk Busuk')
train_wortelbusuk_dir = os.path.join(train_dir, 'Wortel Busuk')
train_tomatsegar_dir = os.path.join(train_dir, 'Tomat Segar')
train_tomatsegarsedang_dir = os.path.join(train_dir, 'Tomat Segar sedang')

# Specify class of the directory test
test_jeruksegar_dir = os.path.join(test_dir, 'Jeruk Segar')
test_wortelsegar_dir = os.path.join(test_dir, 'Wortel Segar')
test_wortelsegarsedang_dir = os.path.join(test_dir, 'Wortel Segar Sedang')
test_tomatbusuk_dir = os.path.join(test_dir, 'Tomat Busuk')
test_jeruksegarsedang_dir = os.path.join(test_dir, 'Jeruk Segar Sedang')
test_jerukbusuk_dir = os.path.join(test_dir, 'Jeruk Busuk')
test_wortelbusuk_dir = os.path.join(test_dir, 'Wortel Busuk')
test_tomatsegar_dir = os.path.join(test_dir, 'Tomat Segar')
test_tomatsegarsedang_dir = os.path.join(test_dir, 'Tomat Segar Sedang')

In [35]:
train_jeruksegar_fnames = os.listdir(train_jeruksegar_dir)
train_wortelsegar_fnames = os.listdir(train_wortelsegar_dir)
train_wortelsegarsedang_fnames = os.listdir(train_wortelsegarsedang_dir)
train_tomatbusuk_fnames = os.listdir(train_tomatbusuk_dir)
train_jeruksegarsedang_fnames = os.listdir(train_jeruksegarsedang_dir)
train_jerukbusuk_fnames = os.listdir(train_jerukbusuk_dir)
train_wortelbusuk_fnames = os.listdir(train_wortelbusuk_dir)
train_tomatsegar_fnames = os.listdir(train_tomatsegar_dir)
train_tomatsegarsedang_fnames = os.listdir(train_tomatsegarsedang_dir)

print(train_jeruksegar_fnames[:10])
print(train_wortelsegar_fnames[:10])

['jeruk (808).png', 'jeruk (326).png', 'jeruk (355).png', 'jeruk (1081).png', 'jeruk (921).png', 'jeruk (155).png', 'jeruk (1121).png', 'jeruk (750).png', 'jeruk (849).png', 'jeruk (993).png']
['1353.jpg', '0963.jpg', '0505.jpg', '1216.jpg', '1013.jpg', '0929.jpg', '0818.jpg', '1139.jpg', '0610.jpg', '1080.jpg']


In [36]:
print('total training jeruk segar images :', len(os.listdir(train_jeruksegar_dir)))
print('total training wortel segar images :', len(os.listdir( train_wortelsegar_dir)))
print('total training wortel segar sedang images :', len(os.listdir(train_wortelsegarsedang_dir)))
print('total training tomat busuk images :', len(os.listdir(train_tomatbusuk_dir)))
print('total training jeruk segar sedang images :', len(os.listdir(train_jeruksegarsedang_dir)))
print('total training jeruk busuk images :', len(os.listdir(train_jerukbusuk_dir)))
print('total training wortel busuk images :', len(os.listdir(train_wortelbusuk_dir)))
print('total training tomat segar images :', len(os.listdir(train_tomatsegar_dir)))
print('total training tomat segar sedang images :', len(os.listdir(train_tomatsegarsedang_dir)))

print('total testing jeruk segar images :', len(os.listdir(test_jeruksegar_dir)))
print('total testing wortel segar images :', len(os.listdir(test_wortelsegar_dir)))
print('total testing wortel segar sedang images :', len(os.listdir(test_wortelsegarsedang_dir)))
print('total testing tomat busuk images :', len(os.listdir(test_tomatbusuk_dir)))
print('total testing jeruk segar sedang images :', len(os.listdir(test_jeruksegarsedang_dir)))
print('total testing jeruk busuk images :', len(os.listdir(test_jerukbusuk_dir)))
print('total testing wortel busuk images :', len(os.listdir(test_wortelbusuk_dir)))
print('total testing tomat segar images :', len(os.listdir(test_tomatsegar_dir)))
print('total testing tomat segar sedang images :', len(os.listdir(test_tomatsegarsedang_dir)))

total training jeruk segar images : 1368
total training wortel segar images : 1114
total training wortel segar sedang images : 106
total training tomat busuk images : 1002
total training jeruk segar sedang images : 693
total training jeruk busuk images : 1280
total training wortel busuk images : 183
total training tomat segar images : 1185
total training tomat segar sedang images : 169
total testing jeruk segar images : 317
total testing wortel segar images : 279
total testing wortel segar sedang images : 40
total testing tomat busuk images : 85
total testing jeruk segar sedang images : 173
total testing jeruk busuk images : 319
total testing wortel busuk images : 45
total testing tomat segar images : 204
total testing tomat segar sedang images : 42
